In [ ]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loader.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langhchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

%pwd

In [ ]:
!mkdir test_repo

In [ ]:
repo_path = "test_repo/"

repo = Repo.clone_from("https://github.com/Eltaf-azizi/auto-code-analyzer.git")

In [ ]:
loader = GenericLoader.from_filesystem(repo_path,
                                       glob = "**/*",
                                       suffixes=[".py"],
                                       parser = LanguageParser(language=Language.PYTHON))

In [ ]:
documents = loader.load()

In [ ]:
documents

In [ ]:
len(documents)

In [ ]:
documents[0]

In [ ]:
documents_splitter = RecursiveCharacterTextSplitter.from_language(language = Language.PYTHON,
                                                                  chunk_size = 500,
                                                                  chunk_overlap = 20)

In [ ]:
texts = documents_splitter.split_documents(documents)

In [ ]:
texts

In [ ]:
len(texts)

In [ ]:
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [ ]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
embeddings = OpenAIEmbeddings(disallowed_special=())

In [ ]:
vectordb = Chroma.from_documents(texts, embedding=embeddings, persist_directory=".db")

In [ ]:
vectordb.persist()

In [ ]:
# llm = ChatOpenAI(model_name="gpt-turbo-3.5")
llm = ChatOpenAI()

In [ ]:
memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)

In [ ]:
qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k"}))

In [ ]:
question = "What is download_hugging_face_embeddings function?"

In [ ]:
result = qa(question)
print(result['answer'])